# 컨텍스트 관리자 (context manager)

~~~python
with open(filename) as fd:
    process_file(fd)
~~~
with 문은 컨텍스트 관리자로 진입하게 한다. 컨텍스트 관리자는
- __enter__
- __exit__
의 두 개의 매직 메소드로 구성된다. with 문은 __enter__ 메소드를 호출하고 as 이후에 지정된 변수에 할당된다.     
해당 블록에 대한 마지막 문장이 끝나면 컨텍스트가 종료되며 이는 파이썬이 처음 호출한 원래 컨텍스트 관리자 객체의 __exit__ 메소드를 호출함을 의미한다.     

컨텍스트 관리자는 관심사를 분리하고 독립적으로 유지되어야하는 코드를 분리하느 좋은 방법이다.     


In [4]:
def stop_database():
    print("systemctl stop postgresql.service")

def start_database():
    print("systemctl start postgresql.service")

class DBHandler:
    def __enter__(self):
        stop_database()
        return self 
    
    def __exit__(self, exc_type, ex_value, ex_traceback):
        start_database()

def db_backup():
    print("pg_dump database")

def main():
    with DBHandler():
        db_backup()

In [5]:
main()

systemctl stop postgresql.service
pg_dump database
systemctl start postgresql.service


__enter__와 __exit__ 매직 메소드만 구현하는 방법이 일반적이지만, contextlib 모듈을 사용하여 보다 쉽게 구현할 수 도 있다.     
먼저 contextmanager 데코레이터를 살펴 보아야 한다. 함수에 contextlib.contextmanager 데코레이터를 적용하면 해당 함수의 코드를 컨텍스트 관리자로 변환한다.    
함수는 제너레이터라는 특수한 함수의 형태여야 하는데 이 함수는 코드의 문장을 __enter__와 __exit__ 매직 메소드로 분리한다.        


In [7]:
import contextlib

@contextlib.contextmanager
def db_handler():
    stop_database()
    yield # yield 문을 사용했으므로 generator 가 된다
    start_database()

with db_handler():
    db_backup



systemctl stop postgresql.service
systemctl start postgresql.service


데코레이터를 적용하면 yield 문 앞의 모든 것은 __enter__ 메소드의 일부처럼 취급된다는 것이다      
yield 문 다음에 오는 모든 것들은 __exit__ 로직으로 볼 수 있다

In [9]:
class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_database()
    
    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_database()

@dbhandler_decorator()
def offline_backup():
    print("pg_dump database")

contextlib.ContextDecorator 이다. 이 클래스는 컨텍스트 관리자 안에서 실행될 함수에 데코레이터를 적용하기 위한 로직을 제공하는 믹스인 클래스이다.      
믹스인 클래스는 다른 클래스에서 필요한 기능만 섞어서 사용할 수 있도록 메소드 만을 제공하는 유틸리티 형태의 클래스이다.        
with 문이 없고 그저 함수를 호출하기만 하면 offline_backup 함수가 컨텍스트 관리자 안에서 자동으로 실행된다.      


In [ ]:
import contextlib 

with contextlib.suppress(DataConversionException):
    parse_data(input_json_or_dict)
    